<center> <h1 style="background-color:seagreen; color:white" >Associadores com FP-Growth com Spark usando PySpark</h1></center>  

**Conceitos**:

> + Suporte: numero de transações que contem todos os itens da transação.
> + Confiança: indica a proporção de vezes que, em uma transação contendo o elemento A, também tem B.
> + Lift: o quanto aumenta a frequência de B com a ocorrência de A


 **Suporte**:
> Não adianta os produtos aparecerem com
frequência juntos, se raramente são comprados!

**Confiança**:
> Não adianta um produto ser muito comprado, se
dificilmente eles aparecem juntos

### FP-Grow

FP-Grow induz árvores, e busca sobreposição destas árvores, onde os itens são
frequentes.



<img src="exemp.jpeg" alt="Associadores com FP-Growth" width="500" height="300">

<center> <h2 style="background-color:DarkKhaki; color:white" >Inicializando o PySpark</h2></center> 

In [4]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("FP-Growth").getOrCreate()

<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Hiper Parâmetros</h3></center> 


> + **minConfidence**: confiança minima para gerar regra. (padrão: 0.8)
> + **minSupport**: suporte minímo para gerar a regra. (padrão: 0.3)




---

<center> <h1 style="background-color:seagreen; color:white" >Aplicando Associadores com FP-Growth no DataSet Íris</h1></center>



<center> <h3 style="background-color:#ECF7AE; color:#3F403D" >Carregando o Cunjunto de Dados Íris</h3></center> 

In [12]:
from pyspark.sql.functions import split

transacoes =  spark.read.text("../Material_do_Curso/Transacoes.txt")
transacoes = transacoes.select(split("value", "\s+").alias("transacoes"))
transacoes.show(truncate=False)

+-------------------------+
|transacoes               |
+-------------------------+
|[Cerveja, Pizza, Sorvete]|
|[Pizza, Sorvete]         |
|[Cerveja, Pizza]         |
|[Cerveja, Pizza, Sorvete]|
|[Cerveja, Pizza]         |
|[Pizza]                  |
+-------------------------+



---


<center> <h3 style="background-color:#ECF7AE; color:#3F403D" > Importação do Módulo do PySpark Para Criação do Modelo de Associadores com FP-Growth</h3></center> 


In [13]:
from pyspark.ml.fpm import FPGrowth 

#### Instanciando Objeto e criando o Modelo

In [14]:
# Instanciando o objeto FP-Growth
obj_fp = FPGrowth(itemsCol="transacoes", minSupport=0.5, minConfidence=0.5)

# Criando o Modelo
model_fp = obj_fp.fit(transacoes)
model_fp.freqItemsets.show()

+----------------+----+
|           items|freq|
+----------------+----+
|       [Sorvete]|   3|
|[Sorvete, Pizza]|   3|
|       [Cerveja]|   4|
|[Cerveja, Pizza]|   4|
|         [Pizza]|   6|
+----------------+----+




---

<center> <h1 style="background-color:seagreen; color:white" >Regras Criadas com suas Confianças e Suportes correspondentes</h1></center> 
<br>


In [17]:
model_fp.associationRules.show()

+----------+----------+------------------+----+------------------+
|antecedent|consequent|        confidence|lift|           support|
+----------+----------+------------------+----+------------------+
| [Cerveja]|   [Pizza]|               1.0| 1.0|0.6666666666666666|
| [Sorvete]|   [Pizza]|               1.0| 1.0|               0.5|
|   [Pizza]| [Sorvete]|               0.5| 1.0|               0.5|
|   [Pizza]| [Cerveja]|0.6666666666666666| 1.0|0.6666666666666666|
+----------+----------+------------------+----+------------------+




<center> <h3 style="background-color:#ECF7AE; color:#3F403D" > Prevendo futuras compras baseado no que a pessoa comprou </h3></center> 

In [18]:
model_fp.transform(transacoes).show(truncate=False)

+-------------------------+------------------+
|transacoes               |prediction        |
+-------------------------+------------------+
|[Cerveja, Pizza, Sorvete]|[]                |
|[Pizza, Sorvete]         |[Cerveja]         |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Cerveja, Pizza, Sorvete]|[]                |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Pizza]                  |[Sorvete, Cerveja]|
+-------------------------+------------------+

